import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')

os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://cloud.google.com/ai/generative-ai")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader

In [4]:
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://cloud.google.com/ai/generative-ai', 'title': 'Generative AI | Google Cloud', 'description': "Create, discover, summarize and automate with Google Cloud's generative AI products and services.", 'language': 'en-US'}, page_content="Generative AI | Google CloudPage ContentsDocumentationTry Gemini 2.5, our most intelligent model now available in Vertex AIGenerative AI on Google CloudBring generative AI to real-world experiences quickly, efficiently, and responsibly, powered by Google’s most advanced technology and models including Gemini. Plus, new customers can start their AI journey today with $300 in free credits.Get started for freeContact salesProducts and servicesExplore tools from Google Cloud that make it easier for developers to build with generative AI and new AI-powered experiences across our cloud portfolio. For more information, view all our AI products.AI platform Gemini modelsGoogle Cloud with GeminiGenerative AI partnersAI for startups 

In [8]:
## load data --> dpcs --> divide into chunks --> text -->  vectors --> Vector Embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://cloud.google.com/ai/generative-ai', 'title': 'Generative AI | Google Cloud', 'description': "Create, discover, summarize and automate with Google Cloud's generative AI products and services.", 'language': 'en-US'}, page_content='Generative AI | Google CloudPage ContentsDocumentationTry Gemini 2.5, our most intelligent model now available in Vertex AIGenerative AI on Google CloudBring generative AI to real-world experiences quickly, efficiently, and responsibly, powered by Google’s most advanced technology and models including Gemini. Plus, new customers can start their AI journey today with $300 in free credits.Get started for freeContact salesProducts and servicesExplore tools from Google Cloud that make it easier for developers to build with generative AI and new AI-powered experiences across our cloud portfolio. For more information, view all our AI products.AI platform Gemini modelsGoogle Cloud with GeminiGenerative AI partnersAI for startups 

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [12]:
vectorstoredb

In [15]:
## Query from a vector db
query = "Dengan Vertex AI, yang merupakan platform pengembangan AI terpadu kami,"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'Vertex AI, our unified AI development platform, you can interact with, customize, and embed foundation models into your applications. Access 200+ foundation models on Model Garden, tune models via a simple UI on Vertex AI Studio, or use models directly in a data science notebook.ResourcesWhat Google Cloud announced in AI this monthRead the blog25+ top gen AI how-to guides for Google CloudRead the blogCustomize and deploy Gemini models to production in Vertex AI Gemini, a multimodal model from Google DeepMind, is capable of understanding virtually any input, combining different types of information, and generating almost any output. Prompt and test Gemini in Vertex AI using text, images, video, or code. With Gemini’s advanced reasoning and generation capabilities, developers can try sample prompts for extracting text from images, converting image text to JSON, and even generate answers about uploaded images.ResourcesVertex AI Gemini API QuickstartDocumentationGemini 2.5 brings enhanced

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-nano-2025-08-07")

In [17]:
## retrieval chain, Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
### pROMPT TEMPLATE
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
        Jawab pertanyaan berdasarkan dengan hanya darri context yang disediakan::
        <context>
            {context}
        </context>
    """
)


document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Jawab pertanyaan berdasarkan dengan hanya darri context yang disediakan::\n        <context>\n            {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024F82131D50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024FBEBAE560>, root_client=<openai.OpenAI object at 0x0000024FF2416440>, root_async_client=<openai.AsyncOpenAI object at 0x0000024F82E2F2E0>, model_name='gpt-5-nano-2025-08-07', model_kwargs={}, openai_api_key=Sec

In [20]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input": "Apa itu gen ai?",
        "context": [Document(page_content="Dengan Vertex AI, yang merupakan platform pengembangan AI terpadu kami, Anda dapat berinteraksi dengan model dasar, menyesuaikan model tersebut, dan menyematkannya ke aplikasi Anda. Anda dapat mengakses lebih dari 200 model dasar di Model Garden, menyesuaikan berbagai model melalui UI yang sederhana di Vertex AI Studio, atau menggunakan model secara langsung dalam notebook data science.")]
    }
)

'Silakan ajukan pertanyaan Anda. Berdasarkan konteks yang diberikan: Vertex AI adalah platform pengembangan AI terpadu di mana Anda dapat berinteraksi dengan model dasar, menyesuaikan model tersebut, dan menyematkannya ke aplikasi Anda. Anda dapat mengakses lebih dari 200 model dasar di Model Garden, menyesuaikan berbagai model melalui UI yang sederhana di Vertex AI Studio, atau menggunakan model secara langsung dalam notebook data science.'

In [21]:
## Retriever

vectorstoredb

In [23]:
retriever = vectorstoredb.as_retriever()

In [24]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024FBD006B30>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Jawab pertanyaan berdasarkan dengan hanya darri context yang disediakan::\n        <context>\n            {context}\n        </context>\n  

In [26]:
## Get response from the LLM

response = retrieval_chain.invoke(
    {
        "input": "Apa itu gen ai?",
    }
)

In [28]:
response['answer']

'Silakan ajukan pertanyaannya. Saya akan menjawab hanya berdasarkan informasi yang ada dalam konteks tersebut.\n\nJika membantu, beberapa topik dari konteks yang bisa menjadi acuan pertanyaan:\n- Apa itu Vertex AI dan bagaimana cara menggunakannya (Model Garden, Vertex AI Studio, notebook)?\n- Apa itu Gemini dan bagaimana fungsinya (multimodal, API Quickstart, Gemini 2.5)?\n- Bagaimana cara bekerja dengan model foundation dan bagaimana integrasinya ke aplikasi?\n- Fitur-fitur utama yang disebutkan seperti AI agents, AI-powered search, dan automasi dengan GenAI.'

In [29]:
response

{'input': 'Apa itu gen ai?',
 'context': [Document(id='afd0340a-1fe3-4d3a-aef6-f0f2c2d5cfe6', metadata={'source': 'https://cloud.google.com/ai/generative-ai', 'title': 'Generative AI | Google Cloud', 'description': "Create, discover, summarize and automate with Google Cloud's generative AI products and services.", 'language': 'en-US'}, page_content='a gen AI application: Design principles and design patterns5-min readUnlock gen AI’s full potential with operational databasesView MoreExecutive resourcesLearn how nonprofits are tackling climate action, education, and crisis response with AIDownload the guideJoin Google experts for a deep dive into how companies are putting AI to workListen to the podcastUnlocking gen AI success: Five pitfalls every executive should knowRead the blogView MoreConsulting servicesCreate with generative AITransform your creative process. Boost productivity by automatically generating writing and art.\xa0Learn moreDiscover with generative AIBuild AI-enhanced se

In [31]:
response["context"]

[Document(id='afd0340a-1fe3-4d3a-aef6-f0f2c2d5cfe6', metadata={'source': 'https://cloud.google.com/ai/generative-ai', 'title': 'Generative AI | Google Cloud', 'description': "Create, discover, summarize and automate with Google Cloud's generative AI products and services.", 'language': 'en-US'}, page_content='a gen AI application: Design principles and design patterns5-min readUnlock gen AI’s full potential with operational databasesView MoreExecutive resourcesLearn how nonprofits are tackling climate action, education, and crisis response with AIDownload the guideJoin Google experts for a deep dive into how companies are putting AI to workListen to the podcastUnlocking gen AI success: Five pitfalls every executive should knowRead the blogView MoreConsulting servicesCreate with generative AITransform your creative process. Boost productivity by automatically generating writing and art.\xa0Learn moreDiscover with generative AIBuild AI-enhanced search engines or assistive experiences to 